## **Modelo CNN Simple - ResNet18 + Dropout + Soft Attention Espacial** 

In [1]:
import torch
import os
import pandas as pd
import seaborn as sns
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from datetime import datetime
from torchvision import transforms
from torch.utils.data import Subset
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from PIL import Image

## Definición de transformación de los datos de entrada

In [2]:
# Definicion de parámetros para la modificación de imágenes
crop_size = (64, 64)
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
rotation_grade = 10

In [3]:
transform = transforms.Compose([
        transforms.Resize(crop_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(rotation_grade),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

## Definición del dataset

In [4]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        """
        Args:
            csv_file (str): Ruta al archivo CSV con las imágenes y sus etiquetas.
            img_dir (str): Ruta al directorio que contiene las imágenes.
            transform (callable, optional): Transformaciones que se aplican a las imágenes.
        """
        self.img_labels = pd.read_csv(csv_file)  # Leer el archivo CSV con las etiquetas
        self.img_dir = img_dir  # Ruta donde están las imágenes
        self.transform = transform  # Transformaciones a aplicar

        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(self.img_labels['diagnosis'].unique())}
    def __len__(self):
        """Retorna el número total de imágenes en el dataset"""
        return len(self.img_labels)

    def __getitem__(self, idx):
        """Obtiene una imagen y su etiqueta"""
        img_name = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])  # Nombre de la imagen
        image = Image.open(img_name)  # Abrir la imagen
        label = self.class_to_idx[self.img_labels.iloc[idx, 3]] # Etiqueta asociada

        if self.transform:
            image = self.transform(image)  # Aplicar transformaciones si es necesario

        return image, label

## Rutas de archivos

In [5]:
# Leer el CSV
csvRoute="./content/GPTeam-DeepLearning/Dataset/bcn_20k_train.csv"
imagesFolderRoute = "./content/GPTeam-DeepLearning/Dataset/bcn_20k_train/"
trainDataJsonRoute = "./content/GPTeam-DeepLearning/Dataset/data_train_resnet18_softAtt.json"
validationDataJsonRoute= "./content/GPTeam-DeepLearning/Dataset/data_val_resnet18_softAtt.json"
filteredCsvRoute = "./content/GPTeam-DeepLearning/Dataset/bcn_20k_train_filtrado.csv"
logsRoute = "./runs"
savedParametersRoute = "./saved_models_parameters/Parameters"
savedModelsRoute = "./saved_models_parameters/Models"
savedResultsRoute = "./results"

## Generación del dataset de entrenamiento y validación

***Carga del dataframe***

In [6]:
df = pd.read_csv(csvRoute)
df.head()

,bcn_filename,age_approx,anatom_site_general,diagnosis,lesion_id,capture_date,sex,split
0,BCN_0000000001.jpg,55.0,anterior torso,MEL,BCN_0003884,2012-05-16,male,train
1,BCN_0000000003.jpg,50.0,anterior torso,MEL,BCN_0000019,2015-07-09,female,train
2,BCN_0000000004.jpg,85.0,head/neck,SCC,BCN_0003499,2015-11-23,male,train
3,BCN_0000000006.jpg,60.0,anterior torso,NV,BCN_0003316,2015-06-16,male,train
4,BCN_0000000010.jpg,30.0,anterior torso,BCC,BCN_0004874,2014-02-18,female,train


***Exclusión de clases menos representativas***

In [7]:
df_original = df.copy()
#Definir las clases que deseas excluir por su nombre
clases_a_excluir = ['SCC', 'DF', 'VASC']  # Sustituye estos nombres por las clases que quieres excluir

# Filtrar el DataFrame para excluir las clases especificadas
df_filtrado = df[~df['diagnosis'].isin(clases_a_excluir)]
df_filtrado.to_csv(filteredCsvRoute, index=False)
df = df_filtrado.copy()
df_filtrado['diagnosis'].unique()

array(['MEL', 'NV', 'BCC', 'BKL', 'AK'], dtype=object)

***Busqueda de spliteo adecuado (sin lesson_id iguales para dataset de entrenamiento y validación)***

In [8]:
# Obtener los valores únicos de lesion_id
unique_lesions = df['lesion_id'].unique()

# Búsqueda del mejor split
best_train, best_test = None, None
target_ratio = 0.8 * len(df)

for seed in range(1000):  # Exploramos diferentes random_state
    train_ids, test_ids = train_test_split(unique_lesions, test_size=0.2, random_state=seed)

    x_train = df[df['lesion_id'].isin(train_ids)]
    x_test = df[df['lesion_id'].isin(test_ids)]

    # Verificar que la cantidad de filas sea la correcta Y que los lesion_id no se repitan
    if abs(len(x_train) - target_ratio) < 1 and set(x_train['lesion_id']).isdisjoint(set(x_test['lesion_id'])):
        best_train, best_test = x_train, x_test
        print(f"Random state encontrado: {seed}")
        break

# Si no se encontró un split válido, lanzar error
if best_train is None or best_test is None:
    raise ValueError("No se encontró un split válido después de 1000 intentos")

# Asignar los mejores valores encontrados
train_df, val_df = best_train, best_test

assert set(train_df['lesion_id']).isdisjoint(set(val_df['lesion_id'])), "Error: Hay lesion_id repetidos entre train_df y val_df"

Random state encontrado: 25


In [9]:
# VERIFICACION ADICIONAL DE NO REPETICION DE LESSIONID EN AMBOS DATASETS
for idtrain in train_df['lesion_id'].unique():
    for idval in val_df['lesion_id'].unique():
        if idtrain.strip() == idval.strip():
            print("Coincidence found")

***Carga de las imágenes***

In [10]:
# Crear el dataset de entrenamiento y validación
train_dataset = CustomDataset(csv_file=filteredCsvRoute, img_dir=imagesFolderRoute, transform=transform)
val_dataset = CustomDataset(csv_file=filteredCsvRoute, img_dir=imagesFolderRoute, transform=transform)

# Actualizar los datasets con los subconjuntos correspondientes
train_dataset.img_labels = train_df
val_dataset.img_labels = val_df

In [11]:
# Verificación de la carga de todos los arhivos
archivos_en_directorio = []
# Recorrer todos los archivos en el directorio
for archivo in os.listdir(imagesFolderRoute):
    if os.path.isfile(os.path.join(imagesFolderRoute, archivo)):
        archivos_en_directorio.append(archivo)

# La columna del DataFrame con los nombres de los archivos
column_files = df['bcn_filename']
# Convertir las listas a conjuntos para realizar la diferencia
set_column_files = set(column_files)
set_archivos_en_drive = set(archivos_en_directorio)
valores_faltantes = set_column_files - set_archivos_en_drive

print(f"Cantidad de imagenes faltantes: {len(valores_faltantes)}")

Cantidad de imagenes faltantes: 0


## **Guardado de los datasets divididos inicialmente**

**Guardado de data de dataframes de entrenamiento y validación**

In [12]:
train_df.to_json(trainDataJsonRoute, orient='records', lines=True)
val_df.to_json(validationDataJsonRoute, orient='records', lines=True)

**Definición del Modelo**

In [13]:
class TransformerModule(nn.Module):
    """
    Módulo transformer para procesar secuencias, usando batch_first.
    """
    def __init__(self, seq_length, d_model=256, nhead=8, num_layers=2, dropout=0.5):
        super(TransformerModule, self).__init__()
        self.pos_embedding = nn.Parameter(torch.randn(seq_length, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: (B, seq_length, d_model)
        x = x + self.pos_embedding.unsqueeze(0)
        x = self.transformer_encoder(x)  # Ahora sin necesidad de transponer
        x = x.mean(dim=1)
        x = self.dropout(x)
        return x

class CNNTransformerClassifier(nn.Module):
    """
    Modelo híbrido que integra una CNN de 4 capas (divididas en 2 bloques con pooling)
    con un módulo Transformer. La salida final es una clasificación en 5 clases.
    Se aplican BatchNorm, Dropout y se inicializan pesos con Kaiming.
    """
    def __init__(self, num_classes=5, dropout=0.5):
        super(CNNTransformerClassifier, self).__init__()
        # Bloque 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # 64x64 -> 32x32

        # Bloque 2
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4   = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # 32x32 -> 16x16

        # Datos para el transformer
        self.feature_map_size = 16  # luego de pool2 (16x16)
        self.seq_length = self.feature_map_size * self.feature_map_size  # 256 tokens
        self.d_model = 256  # dimensión de cada token

        # Módulo Transformer
        self.transformer = TransformerModule(seq_length=self.seq_length, d_model=self.d_model, nhead=8, num_layers=2, dropout=dropout)

        # Capa final para clasificación
        self.fc = nn.Linear(self.d_model, num_classes)
        self.dropout = nn.Dropout(dropout)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # Bloque 1
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        # Bloque 2
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)  # x: (B, 256, 16, 16)

        B, C, H, W = x.size()
        # Reorganizar para formar secuencia: (B, tokens, d_model)
        x = x.view(B, C, -1)    # (B, 256, 256)
        x = x.transpose(1, 2)   # (B, 256, 256)

        # Procesar con el Transformer
        x = self.transformer(x)  # (B, d_model)
        x = self.dropout(x)
        logits = self.fc(x)
        return logits


# Visualización de la ejecución del modelo

Ejecutar desde cmd lo siguiente para ver desde el navegador el menú de tensorboard: tensorboard --logdir=runs

In [14]:
%load_ext tensorboard
%tensorboard --logdir runs

Reusing TensorBoard on port 6007 (pid 22044), started 1 day, 1:32:49 ago. (Use '!kill 22044' to kill it.)

## **Entrenamiento y validación del modelo**

**Definición de gráficas adicionales a emplear**

In [15]:
# Función para registrar el heatmap (matriz de confusión)
def log_heatmap(writer, epoch, ground_truth, predictions, labels):
    # Calcula la matriz de confusión usando el orden de las etiquetas
    cm = confusion_matrix(ground_truth, predictions, labels=range(len(labels)))
    
    # Crear la figura
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=labels, yticklabels=labels, ax=ax)
    ax.set_xlabel("Predicción")
    ax.set_ylabel("Groundtruth")
    ax.set_title("Matriz de Confusión")
    
    # Registrar la figura en TensorBoard
    writer.add_figure("Heatmap", fig, global_step=epoch)
    plt.close(fig)

# Función para registrar la grilla 2x2 de TP, FP, TN, FN (para clasificación binaria)
def log_values_predicted(writer, epoch, ground_truth, predictions):
    # Calcular la matriz de confusión
    cm = confusion_matrix(ground_truth, predictions, labels=[0, 1])
    
    # Crear anotaciones combinando etiqueta y valor
    annot = np.array([
        [f"TN\n{cm[0,0]}", f"FP\n{cm[0,1]}"],
        [f"FN\n{cm[1,0]}", f"TP\n{cm[1,1]}"]
    ])
    
    # Crear el heatmap
    fig, ax = plt.subplots(figsize=(6, 5))
    sns.heatmap(cm, annot=annot, fmt="", cmap="Blues",
                xticklabels=["False", "True"],
                yticklabels=["False", "True"],
                cbar=True, ax=ax)
    ax.set_xlabel("Eje Predicho (False/True)")
    ax.set_ylabel("Eje Verdadero (False/True)")
    ax.set_title("Predicción de valores")
    
    # Registrar la figura en TensorBoard
    writer.add_figure("Predicción de valores", fig, global_step=epoch)
    plt.close(fig)

**Seteo del dispositivo a usar**

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print(F"Dispositivo utilizado: {device} ({torch.cuda.get_device_name()})")
else:
    print(F"Dispositivo utilizado: {device}")

Dispositivo utilizado: cuda (NVIDIA GeForce RTX 2060)


**Especificación de parámetros**

In [17]:
learning_rate = 0.001
num_classes = 5
num_epochs = 120
batch_size = 64
dropout = 0.5

In [18]:
# Inicialización de tensorboard
executionModelDateTime = datetime.now().strftime("Ejecucion %d-%m-%Y %H-%M")
writer = SummaryWriter(log_dir=os.path.join(logsRoute,executionModelDateTime))

# Crear los DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Definición de la cantidad de clases, la función de perdida, el optimizador y el learning rate estático
modelAt = CNNTransformerClassifier(num_classes=num_classes, dropout=dropout)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modelAt.parameters(), lr=learning_rate)
model = modelAt.to(device)
executionStatistics_list = []

for epoch in range(num_epochs):
    model.train()  # Modo entrenamiento
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    # Entrenamiento
    for inputs, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} - Training", leave=True):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Estadísticas de la pérdida
        running_loss += loss.item()
        # Precisión
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    #print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_dataloader)}, Accuracy: {100 * correct_preds / total_preds}%")
    train_loss = running_loss / len(train_dataloader)
    train_accuracy = 100 * correct_preds / total_preds

    # Validación
    model.eval()  # Modo evaluación
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    all_groundtruth = []
    all_predictions = []

    with torch.no_grad():  # No calcular gradientes durante la validación
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)

            all_groundtruth.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    val_loss = running_val_loss/len(val_dataloader)
    val_accuracy = 100 * correct_preds / total_preds
    
    
    # Agregar valores a TensorBoard
    writer.add_scalar("Loss/Train", train_loss, epoch)
    writer.add_scalar("Loss/Validation", val_loss, epoch)
    writer.add_scalar("Accuracy/Train", train_accuracy, epoch)
    writer.add_scalar("Accuracy/Validation", val_accuracy, epoch)

    # Obtener los nombres de las clases (suponiendo que están en df_filtrados.columns)
    labels_names = list(df_filtrado['diagnosis'].unique())
    # Registrar el heatmap y la grilla de valores en TensorBoard
    log_heatmap(writer, epoch, all_groundtruth, all_predictions, labels_names)
    log_values_predicted(writer, epoch, all_groundtruth, all_predictions)
    
    # Carga de hiperparametros y metricas
    execution_statistics = {
        "epoch": epoch+1,
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "num_epochs": num_epochs,
        "Train Loss": train_loss,
        "Validation Loss": val_loss,
        "Train Accuracy": train_accuracy,
        "Validation Accuracy": val_accuracy
        
    }
    
    executionStatistics_list.append(execution_statistics)
    
    # Gráfica de loss y accuracy por cada epoca
    writer.add_scalars("Loss", {"Train": train_loss, "Validation": val_loss}, epoch)
    writer.add_scalars("Accuracy", {"Train": train_accuracy, "Validation": val_accuracy}, epoch)
    
    
    print(f"Epoch {epoch+1}/{num_epochs} FINISHED => "
        f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
        f"Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%")
    


writer.flush()
writer.close()

Epoch 1/120 - Training: 100%|██████████| 147/147 [03:59<00:00,  1.63s/it]


Epoch 1/120 FINISHED => Train Loss: 1.5520, Val Loss: 1.2852, Train Acc: 42.05%, Val Acc: 44.57%


Epoch 2/120 - Training: 100%|██████████| 147/147 [01:56<00:00,  1.26it/s]


Epoch 2/120 FINISHED => Train Loss: 1.2076, Val Loss: 1.1496, Train Acc: 54.61%, Val Acc: 55.07%


Epoch 3/120 - Training: 100%|██████████| 147/147 [01:49<00:00,  1.34it/s]


...

Epoch 118/120 FINISHED => Train Loss: 1.0189, Val Loss: 1.3376, Train Acc: 60.89%, Val Acc: 59.90%


Epoch 119/120 - Training: 100%|██████████| 147/147 [02:47<00:00,  1.14s/it]


Epoch 119/120 FINISHED => Train Loss: 1.0376, Val Loss: 1.4104, Train Acc: 60.58%, Val Acc: 59.62%


Epoch 120/120 - Training: 100%|██████████| 147/147 [02:50<00:00,  1.16s/it]


Epoch 120/120 FINISHED => Train Loss: 1.0230, Val Loss: 1.3244, Train Acc: 60.84%, Val Acc: 60.27%


# Guardado de los resultados del modelo

In [24]:
df_statistics = pd.DataFrame.from_dict(executionStatistics_list)
df_statistics.to_excel(os.path.join(savedResultsRoute, f"{executionModelDateTime}_results.xlsx"), index=False)

# Agregado de Hiperparámetros

# Guardado del modelo

**Guardado del modelo completo**

In [25]:
torch.save(model, os.path.join(savedModelsRoute, f"{executionModelDateTime}_modelo_entrenado_cnn_transformers_ka_completo.pth"))

**Guardado de los pesos del modelo**

In [26]:
torch.save(model.state_dict(), os.path.join(savedModelsRoute, f"{executionModelDateTime}_modelo_entrenado_cnn_transformers_ka_pesos.pth"))

In [27]:
# Mapeo de clases
print(train_dataset.class_to_idx)

{'MEL': 0, 'NV': 1, 'BCC': 2, 'BKL': 3, 'AK': 4}


# Prueba del modelo

In [28]:
model.eval()  # Ponemos el modelo en modo de evaluación

# Paso 3: Cargar la imagen y aplicar las transformaciones
image_path = 'ka.jpg'  # Pon aquí la ruta de tu imagen
image = Image.open(image_path)  # Abrir la imagen
image_tensor = transform(image)  # Aplicar las transformaciones

image_tensor = image_tensor.unsqueeze(0)  # Convertirlo a un batch de tamaño 1

# Paso 5: Mover la imagen al dispositivo (GPU o CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_tensor = image_tensor.to(device)
model = model.to(device)

# Paso 6: Realizar la predicción
with torch.no_grad():  # No necesitamos gradientes para la inferencia
    output = model(image_tensor)

# Paso 7: Convertir las predicciones en probabilidades con softmax
probabilities = torch.nn.functional.softmax(output, dim=1)  # Usamos dim=1 porque tenemos un batch

# Paso 8: Obtener la clase con la mayor probabilidad
_, predicted_class = torch.max(probabilities, dim=1)

# Paso 9: Interpretar la clase predicha
# Usamos el mapeo que ya tienes de clases (el 'class_to_idx' que ya definiste en tu dataset)
predicted_idx = predicted_class.item()  # Obtenemos el índice de la clase predicha
print(predicted_idx)
# Aquí usamos el mapeo de clases que creamos antes para convertir el índice a una clase legible
predicted_class_name = [key for key, value in train_dataset.class_to_idx.items() if value == predicted_idx][0]

# Mostrar la clase predicha
print(f"Predicción: {predicted_class_name}")

2
Predicción: BCC
